In [1]:
import numpy as np
import pandas as pd
import re
import ast
import matplotlib.pyplot as plt
#!pip install wordcloud


In [24]:
path = input('Enter path to master dataset')

In [23]:
df = pd.read_csv (path) 

In [21]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'USN', '10thPercentage', '10thMarks',
       '12thPercentage', 'Branch', 'CGPA', 'CourseName', 'CourseGrade',
       'CompanyName_New', 'CTC', 'TierLevel', 'CoCurricularActivities',
       'EmploymentType', 'EventsParticipated', 'GeneralSkills', 'Languages',
       'MinorAttended', 'NoofInternships', 'InternshipProjectDomain',
       'InternshipCompany', 'NoofProjects', 'ProjectDetailDomain',
       'ProgLanguages', 'Publication', 'ResearchDomain', 'ScholarshipsKey',
       'SoftwareTools', 'VolunteeringWork', 'WorkshopsOrg', 'WorkshopsDomain',
       'AwardsNLPkey', 'ExternalCertificatesKey',
       'ExternalCertificatesDomain'],
      dtype='object')

In [4]:
df_courses = df[['CompanyName_New','CourseName','CourseGrade']]

In [5]:
df_courses.CompanyName_New.isnull().value_counts()

False    180
True      13
Name: CompanyName_New, dtype: int64

In [6]:
df_courses = df_courses[df_courses['CompanyName_New'].notna()]

In [7]:
#---------------------------------------------regularizing subjects -----------------------------------------------------------
# done'unified_engineering','unified_engineering_(jedi)',
# done'engineering_mechanics','engineering_mechanics-dynamics',
# done 'introduction_to_data_science','introduction_to_data_sciences',
# done  __ to _
# done system -> systems
# done enginering -> engineering
# done dbms -> data_base_management_systems
# done -  to -
# done & to and
# done metal_casting_and_welding', 'metal_casting_and_welding_processes',
# done 'power_electronics', 'power_electronics_and_its_applications_to_converters',
# done 'web_technologies-i','web_technologies-ii', 'web_technologies_i', 'web_technologies_ii', 'web_technology-_i',
# 'wireless_communication','wireless_network_communications'
# done processors to processor 
def regularize_courses(l):
    for i in range(0,len(l)):
        l[i] = l[i].replace('-',' ')
        #l[i] = re.sub(' *- *', '-', l[i])
        l[i] = re.sub(' +',' ',l[i])
        l[i] = l[i].replace('&','and').replace('systems','system').replace('sciences','science').replace('processors','processor').replace('technology','technologies').replace('enginering','engineering').replace('1','i').replace('2','ii')
        l[i]= l[i].strip().replace(' ','_')
        if(l[i].count('laboratory')>0):
            l[i] = l[i].replace('laboratory','').strip('_')
        if(l[i]=='unified_engineering_(jedi)'):
            l[i]= 'unified_engineering'
        if(l[i]=='wireless_communication'):
            l[i]= 'wireless_network_communications'
        if(l[i] == 'engineering_mechanics_dynamics'):
            l[i] = 'engineering_mechanics'
        if(l[i].count('dbms')>0):
            l[i] = l[i].replace('dbms','data_base_management_systems')
        if(l[i].count('metal_casting_and_welding')>0):
            l[i] = 'metal_casting_and_welding'
        if(l[i].count('power_electronics')>0):
            l[i] = 'power_electronics'
        if(l[i].count('mini_project')>0):
            l[i] = 'mini_project'
        if(l[i].count('vlsi')>0):
            l[i] = 'vlsi'
        if(l[i].count('project_work')>0):
            l[i] = 'project_work'
    l_unique = list(np.unique(np.array(l)))
    l = '##'.join(l_unique)
    return l
        

In [8]:
df_courses['CourseName_New'] = df.apply(lambda row : regularize_courses( list(ast.literal_eval(row['CourseName']))), axis = 1) 

In [9]:
#getting distinct list of all courses, after cleaning
course = df_courses.CourseName_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).to_frame()
course_all = course.index.values.tolist()
#print(len(sorted(course_all)))
#sorted(course_all)

201


['adaptive_system',
 'advanced_algorithms',
 'advanced_computer_networks',
 'advanced_data_base_management_system',
 'advanced_digital_image_processing',
 'advanced_machine_learning',
 'advanced_problem_solving_skills_for_software_engineers',
 'aerospace_structures',
 'algorithms_for_information_retrieval',
 'analog_circuit_design',
 'analog_electronic_circuits',
 'android_application_development',
 'applied_cryptography',
 'artificial_intelligence',
 'artificial_neural_networks',
 'basic_electrical_engineering',
 'basic_electronics_engineering',
 'basic_of_web_development',
 'basics_of_aerospace_engineering',
 'basics_of_automobile_engineering',
 'big_data',
 'blockchain',
 'blockchain_architecture_design_and_use_cases',
 'c++',
 'chemistry',
 'cie_level_i',
 'cie_level_ii',
 'classical_cryptoanalysis',
 'cloud_computing',
 'communication_engineering',
 'compiler_design',
 'computer_aided_engineering_graphics',
 'computer_forensics',
 'computer_network_security',
 'computer_networks',

In [10]:
# removing courses that appear in more than 75% of the data, since they will mostly be core courses
course = df_courses.CourseName_New.apply(lambda x: pd.value_counts(x.split("##"))).sum(axis = 0).lt(0.75*df_courses.shape[0]).to_frame()
course_distinct = course[course[0] == True]
course_distinct = course_distinct.index.values.tolist()
#print(len(course_distinct), course_distinct)

In [11]:
# function to find occurence of specific courses and creating new column for each unqiue course. Fill with 1/0 based on if course was taken or not
def keyword_matching_course(keywords,df):
    df = df.join(pd.DataFrame(0, columns=keywords, index=df.index))
    df['CourseNameKey'] = ''
    print(df.shape)
    for key in keywords:
        #print(key)
        for i in range(0, len(df)):
            if(df.iloc[i]['CourseName_New'].find(key)!=-1):
                df.iloc[i, df.columns.get_loc('CourseNameKey')] =df.iloc[i, df.columns.get_loc('CourseNameKey')]+ key+'##'
                df.iloc[i, df.columns.get_loc(key)] = 1
    return df

In [12]:
df_courses = keyword_matching_course(course_distinct,df_courses)

(180, 174)


In [13]:
# getting total number of entries per company, and total sum of each course for each company
dfCourseCount = df_courses.groupby(['CompanyName_New'])[course_distinct].apply(lambda x : x.astype(int).sum())
dfCompanyCount = df_courses.groupby(['CompanyName_New'])['CourseName'].count().to_frame().rename(columns = {'CourseName': 'Total'}, inplace = False)

In [14]:
result = pd.concat([dfCompanyCount, dfCourseCount], axis=1)

In [15]:
# getting a rating by calculating, total number of students placed at a specific company who took that course/ total number of students placed at that company
result = result[course_distinct].div(result.Total, axis=0)
#result = result.reset_index(level=['CompanyName_New'])

In [16]:
originalRatingsData = result.stack().reset_index().rename(columns={'level_0':'CompanyName','level_1':'CourseName', 0:'Rating'})